### R의 코드를 좀 더 효율적으로 짜는 아주 간단한 방법

* Pre-Allocation Memory
* Compiler Bytecode
* Vectorise
* Check for condition
* Using ifelse
* Using which

### Pre-Allocation Memory

In [1]:
# 1. 단순히 결과를 이어붙이는 경우

f1 <- function(n){
  output <- c()
  for(i in 1:n){
    output <- c(output,i)
  }
  return(output)
}
f1(10)

[1]  1  2  3  4  5  6  7  8  9 10

In [2]:
# 2. 결과값을 인덱싱하여 집어넣는 경우
f2 <- function(n){
  output <- c()
  for(i in 1:n){
    output[i] <- i
  }
  return(output)
}
f2(10)

[1]  1  2  3  4  5  6  7  8  9 10

In [3]:
system.time(f1(50000))
system.time(f2(50000))

   user  system elapsed 
   2.61    0.03    2.66 

   user  system elapsed 
   0.02    0.00    0.01 

In [4]:
# 3. output의 길이를 미리 지정하고 인덱싱하여 집어넣는 경우 (Pre-Allocation)
f3 <- function(n){
  output <- numeric(n)
  for(i in 1:n){
    output[i] <- i
  }
  return(output)
}
f3(10)

[1]  1  2  3  4  5  6  7  8  9 10

In [6]:
system.time(f1(500000))
system.time(f2(500000))
system.time(f3(500000))

   user  system elapsed 
 271.27    1.96  276.36 

   user  system elapsed 
   0.14    0.00    0.14 

   user  system elapsed 
   0.04    0.00    0.04 

* 벡터, 데이터프레임 등에 데이터를 저장할때는 가능하면 인덱싱을 해서 집어넣자
* 결과값의 길이가 사전적으로 정해져있는 데이터라면 미리 길이까지 지정하자

### Compiler Bytecode

In [7]:
f3(10)
library(compiler)
cmp <- cmpfun(f3)

[1]  1  2  3  4  5  6  7  8  9 10

In [8]:
f3

function(n){
  output <- numeric(n)
  for(i in 1:n){
    output[i] <- i
  }
  return(output)
}
<bytecode: 0x0000000025c14fa0>

In [9]:
cmp

function(n){
  output <- numeric(n)
  for(i in 1:n){
    output[i] <- i
  }
  return(output)
}
<bytecode: 0x0000000018051af8>

In [18]:
system.time(f3(10^8))
system.time(cmp(10^8))

   user  system elapsed 
   6.75    0.08    6.86 

   user  system elapsed 
   6.78    0.03    6.81 

* 컴파일러를 지정하면 일반적으로 속도가 향상한다고 한다.
* 다만 이 방법은 모든 경우에 통하는 것은 아닌 것으로 보인다

### Vectorise

In [20]:
col1 <- runif (12^4, 0, 2)
col2 <- rnorm (12^4, 0, 2)
col3 <- rpois (12^4, 3)
col4 <- rchisq (12^4, 2)
df <- data.frame (col1, col2, col3, col4)

In [21]:
# Before vectorise
system.time(
  for(i in 1:nrow(df)){
    if((df[i,1]+df[i,2]+df[i,3]+df[i,4])>4){
      df[i,5] <- 'greater than 4'
    }else{
      df[i,5] <- 'lesser than 4'
    }
  }
)

   user  system elapsed 
   2.17    0.02    2.19 

In [28]:
# After vectorise
df <- df[,1:4]
output <- c()
system.time(
  for(i in 1:nrow(df)){
    if((df[i,1]+df[i,2]+df[i,3]+df[i,4])>4){
      output[i] <- 'greater than 4'
    }else{
      output[i] <- 'lesser than 4'
    }
  }
)

   user  system elapsed 
   0.99    0.00    1.00 

In [29]:
# Vectorise + Pre-Allocation
output <- character(nrow(df))
system.time(
  for(i in 1:nrow(df)){
    if((df[i,1]+df[i,2]+df[i,3]+df[i,4])>4){
      output[i] <- 'greater than 4'
    }else{
      output[i] <- 'lesser than 4'
    }
  }
)

   user  system elapsed 
   0.96    0.00    1.00 

### Check for condition

In [30]:
condition <- (df[,1]+df[,2]+df[,3]+df[,4])>4
output <- character(nrow(df))
system.time(
  for(i in 1:nrow(df)){
    if(condition[i]){
      output[i] <- 'greater than 4'
    }else{
      output[i] <- 'lesser than 4'
    }
  }
)

   user  system elapsed 
   0.01    0.00    0.01 

### Using ifelse

In [31]:
condition <- (df[,1]+df[,2]+df[,3]+df[,4])>4
system.time(
  output <- ifelse(condition,'greater than 4','lesser than 4')
)

   user  system elapsed 
   0.02    0.00    0.01 

### Using which

In [32]:
want <- which(rowSums(df)>4)
output <- rep("lesser than 4", nrow(df))
system.time(
  output[want] <- 'greater than 4'
)

   user  system elapsed 
      0       0       0 